In [3]:
from mads_datasets import DatasetFactoryProvider, DatasetType

from mltrainer.preprocessors import BasePreprocessor
from mltrainer import imagemodels, Trainer, TrainerSettings, ReportTypes, metrics
import mymodels

import torch.optim as optim
import gin

In [4]:
gin.parse_config_file("config.gin")

ParsedConfigFileIncludesAndImports(filename='config.gin', imports=['gin.torch.external_configurables'], includes=[])

In [5]:
preprocessor = BasePreprocessor()
fashionfactory = DatasetFactoryProvider.create_factory(DatasetType.FASHION)
streamers = fashionfactory.create_datastreamer(batchsize=32, preprocessor=preprocessor)
train = streamers["train"]
valid = streamers["valid"]
trainstreamer = train.stream()
validstreamer = valid.stream()

2024-11-03 15:47:41.829 | INFO     | mads_datasets.base:download_data:121 - Folder already exists at /home/asanchezsa/.cache/mads_datasets/fashionmnist
2024-11-03 15:47:41.829 | INFO     | mads_datasets.base:download_data:124 - File already exists at /home/asanchezsa/.cache/mads_datasets/fashionmnist/fashionmnist.pt


In [6]:
print(gin.config_str())

import gin.torch.external_configurables

# Parameters for MyCNN:
# ==============================================================================
MyCNN.dropout_rate = 0.1
MyCNN.filter1 = 32
MyCNN.filter2 = 64
MyCNN.kernel_size = 3
MyCNN.maxpoolwindow = 2
MyCNN.num_classes = 10



In [7]:
accuracy = metrics.Accuracy()

In [9]:
model = mymodels.MyCNN()
model

MyCNN(
  (convolutions): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (dense): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=1152, out_features=128, bias=True)
    (2): ReLU()
    (3): Linear(in_f

In [ ]:
import torch
gin.parse_config_file("config.gin")

units = [256, 128, 64]
loss_fn = torch.nn.CrossEntropyLoss()

settings = TrainerSettings(
    epochs=5,
    metrics=[accuracy],
    logdir="modellogs",
    train_steps=len(train),
    valid_steps=len(valid),
    reporttypes=[ReportTypes.TENSORBOARD, ReportTypes.GIN],
)

for unit1 in units:
    for unit2 in units:
        # gin.bind_parameter("CNN.filter1", unit1)
        # gin.bind_parameter("CNN.filter2", unit2)

        model = mymodels.MyCNN()
        trainer = Trainer(
            model=model,
            settings=settings,
            loss_fn=loss_fn,
            optimizer=optim.Adam,
            traindataloader=trainstreamer,
            validdataloader=validstreamer,
            scheduler=optim.lr_scheduler.ReduceLROnPlateau
        )
        print(f"Training with {unit1} and {unit2}")
        trainer.loop()
        print("Finished training loop")

In [ ]:
import torch
import torch.optim as optim
from mymodels import MyCNN

gin.parse_config_file("config.gin")

loss_fn = torch.nn.CrossEntropyLoss()

settings = TrainerSettings(
    epochs=5,
    metrics=[accuracy],
    logdir="modellogs",
    train_steps=len(train),
    valid_steps=len(valid),
    reporttypes=[ReportTypes.TENSORBOARD, ReportTypes.GIN],
)

# Initialize the model
model = MyCNN(num_classes=10)

# Define other components like loss function, optimizer, etc.
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Example training loop
for epoch in range(5):
    model.train()
    for data, target in trainstreamer:
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()

In [11]:
import torch
import torch.nn as nn

# Example input tensor with shape (batch_size, channels, height, width)
input_tensor = torch.randn(16, 32, 28, 28)

# BatchNorm2d layer
batch_norm = nn.BatchNorm2d(32)

# Apply batch normalization
output_tensor = batch_norm(input_tensor)

print(output_tensor.shape)  # Should be the same shape as input_tensor


torch.Size([16, 32, 28, 28])
